In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
from exp import nb_anchors_loss_metrics_std
from exp import nb_resnet_ssd_std

In [ ]:
import sys 
sys.path.append("..") 
from detect_symbol.exp import nb_databunch
from detect_symbol.exp import nb_resnet_ssd
from detect_symbol.exp import nb_anchors_loss_metrics

In [ ]:
sys.path.append("../detect_symbol") 
from detect_symbol.exp import nb_interpretation

In [ ]:
from detect_symbol.exp.nb_anchors_loss_metrics import get_y, idx_fromScoresHits

In [ ]:
from detect_symbol.exp.nb_interpretation import draw_rect, draw_text, draw_outline, idx_from_gt

In [ ]:
# export
import torch

In [ ]:
# export
from torch import tensor

In [ ]:
from fastai.basic_train import Learner

In [ ]:
# export
from matplotlib import pyplot as plt

In [ ]:
# export
from matplotlib import patches,patheffects

In [ ]:
# export
from IPython.core import debugger as idb

In [ ]:
from fastai.vision.image import Image

In [ ]:
from functools import partial

In [ ]:
# export
from math import ceil

# functions

### show_gt

In [ ]:
def show_gt(x,y):
    img = data.denorm(x)[0].permute(1,2,0)

    _,ax = plt.subplots(1,1,figsize=(15,15))
    ax.imshow(img)
    
    for bb,cat in zip(y[0][0],y[1][0]):
        bb = (bb+1)/2*776
        draw_rect(ax,bb,lw=1)
        
        cat = cat-1
        clas = classes[cat]
        draw_text(ax,bb[[3,0]],clas[:3],sz=10)

### show_gt_anchors

In [ ]:
def show_gt_anchors(x,y,device,n=10):
    '根据ground truch找到负责idxs，绘制这些idxs对应的anchor'
    _,ax = plt.subplots(1,1,figsize=(20,20))
    img = data.denorm(x)[0].permute(1,2,0)
    ax.imshow(img)
    #import pdb; pdb.set_trace()    
    idxs = idx_from_gt(*y,gaf=gaf)
    for idx in idxs:
        gv = gvs[idx]*x.shape[-1]
        draw_rect(ax,gv,lw=1,color='g')
        
        cx = gv[[0,2]].mean()
        cy = gv[[1,3]].mean()
        
        av = avs[idx]*x.shape[-1]
        av = av+tensor([cx,cy,cx,cy]).to(device)
        draw_rect(ax,av,lw=1)
        

        draw_text(ax,av[[3,0]],f'{idx}',sz=10)
    return idxs

###  idx_from_gt

In [ ]:
def idx_from_gt(*gt_batch, gaf):
    loss = 0
    cnt = 0
    for gt_bboxs in gt_batch[0]:
        keep = get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        gt_bboxs = gt_bboxs[keep]
        
        gt_bboxs = (gt_bboxs + 1) / 2
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = idx_fromScoresHits(scores,hits)
        
        return idx

# test

## 准备好learner

In [ ]:
device = torch.device('cuda')

In [ ]:
#先用detect_symbol的数据集测试
data = nb_databunch.get_databunch(data_root = '../detect_symbol/data/ds_20200429', bs=4, device=device)
x,y = data.one_batch()

In [ ]:
gaf = nb_anchors_loss_metrics_std.GridAnchor_Funcs_std(fig_hw = (776,776)
                         , grids = [(49,49)]
                         , device = device)
gvs, avs = gaf.gvs, gaf.avs

In [ ]:
model = nb_resnet_ssd_std.get_resnet18_1ssd_std(num_classes = 17)

In [ ]:
fn = '../detect_symbol_bak20200427/data/ds_20200227/models/run_0.pth'
stat = torch.load(fn)

In [ ]:
for n, p in model.named_parameters():
    print(n, p.shape, p.sum().cpu().detach().numpy(), stat['model'][n].sum().cpu().detach().numpy())
    #print(n, p.shape)
    p = stat['model'][n]

In [ ]:
learn = Learner(data,model)

In [ ]:
# 把模型放到device上，如果使用GPU，则设置使用哪几个GPU
if hasattr(learn.model, "module"): 
    learn.model = model.module

learn.model.to(device);

if device.type=='cuda':
    learn.model = torch.nn.DataParallel(model,device_ids=[0,1])

In [ ]:
learn.model.eval();

## 验证模型的整体表现

In [ ]:
data.train_dl.batch_size


In [ ]:
metrics = [partial(nb_anchors_loss_metrics.clas_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.cent_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.pConf_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.nConf_L,gaf=gaf),
           partial(nb_anchors_loss_metrics.clas_acc,gaf=gaf),
           partial(nb_anchors_loss_metrics_std.cent_d_std,gaf=gaf)]

learn.loss_func = partial(nb_anchors_loss_metrics_std.yolo_L_std, gaf=gaf)

res = learn.validate(data.train_dl,metrics=metrics)

res

## 观察

In [ ]:
classes = data.train_ds.y.classes[1:]
classes

In [ ]:
x,y = next(iter(data.valid_dl))
show_gt(x,y)

In [ ]:
show_gt_anchors(x,y, device)

# export

In [ ]:
!python notebook2script.py --fname 'interpretation.ipynb' --outputDir './exp/'